In [1]:
#Set the modules and bring in the data
import csv
import os
import pandas as pd
import numpy as np
import functools
#Join Conditions
def conjunction(*conditions):
    return functools.reduce(np.logical_and, conditions)
#Kelvin to Fahrenheit conversion
def kelvin_to_fahrenheit(tempereture_kelvin):
   tempereture_fahrenheit = (((tempereture_kelvin - 273) * 9/5) + 32)
   return tempereture_fahrenheit

os.chdir('/Users/macbook/Desktop/Data-Program-Files/Group-Projects/Team-Folder/Raw_Data/')

Trip_Data = pd.read_csv("fhv_tripdata_2017-07.csv", parse_dates=[0], low_memory=False)
Weather_Data = pd.read_csv('NYC_Weather.csv', parse_dates=[0], low_memory=False)
Zip_Data = pd.read_csv("fhv_zone_lookup_zipcode.csv", low_memory=False)

Trip_Data.head()


,Dispatching_base_num,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID
0,B00001,2017-06-01 00:04:00,2017-06-07 01:55:36,NaN,NaN
1,B00001,2017-06-01 00:47:00,2017-06-09 01:25:40,NaN,NaN
2,B00001,2017-06-01 00:07:00,2017-06-15 00:52:17,NaN,NaN
3,B00001,2017-06-01 00:30:00,2017-06-03 02:18:52,NaN,NaN
4,B00001,2017-06-01 00:45:00,2017-06-10 01:49:42,NaN,NaN


In [2]:
#Cleaning the trip data file
Trip_Data = Trip_Data.dropna(subset= ['Pickup_DateTime'])
Trip_Data = Trip_Data.dropna(subset= ['DropOff_datetime'])
Trip_Data = Trip_Data.dropna(subset= ['DOlocationID'])
Trip_Data = Trip_Data.dropna(subset= ['PUlocationID'])
Trip_Data = Trip_Data.fillna(0)

is_not_0_p =  Trip_Data['PUlocationID']!=0 #no location 
is_not_264_p =  Trip_Data['PUlocationID']!=264 #unknown location
is_not_265_p =  Trip_Data['PUlocationID']!=265 #unknown location

is_not_0_d =  Trip_Data['DOlocationID']!=0 #no location 
is_not_264_d =  Trip_Data['DOlocationID']!=264 #unknown location
is_not_265_d =  Trip_Data['DOlocationID']!=265 #unknown location

Trip_Data = Trip_Data[conjunction(is_not_0_p,is_not_264_p,is_not_265_p,is_not_0_d,is_not_264_d,is_not_265_d)] #clean data

Trip_Data['Pickup_DateTime'] = pd.to_datetime(Trip_Data['Pickup_DateTime'])
Trip_Data['DropOff_datetime'] = pd.to_datetime(Trip_Data['DropOff_datetime'])
Trip_Data['PUlocationID'] = pd.to_numeric(Trip_Data['PUlocationID'])
Trip_Data['DOlocationID'] = pd.to_numeric(Trip_Data['DOlocationID'])
Trip_Data['Pickup_DateTime2'] = Trip_Data.Pickup_DateTime.dt.round('H')

Trip_Data.head()


,Dispatching_base_num,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID,Pickup_DateTime2
17,B00021,2017-06-01 00:00:15,2017-06-01 00:05:42,173.0,173.0,2017-06-01 00:00:00
19,B00021,2017-06-01 00:00:54,2017-06-01 00:07:00,56.0,56.0,2017-06-01 00:00:00
20,B00021,2017-06-01 00:45:37,2017-06-01 01:01:52,260.0,82.0,2017-06-01 01:00:00
21,B00021,2017-06-01 00:41:31,2017-06-01 01:27:57,196.0,56.0,2017-06-01 01:00:00
22,B00021,2017-06-01 00:23:36,2017-06-01 00:34:24,56.0,56.0,2017-06-01 00:00:00


In [3]:
#Getting random sample from the Trip Data file
Trip_Data_Sample = Trip_Data.sample(85000, replace=True)

Trip_Data_Sample = Trip_Data_Sample.sort_values(by=['Pickup_DateTime'])

Trip_Data_Sample = Trip_Data_Sample.reset_index() 

Trip_Data_Sample.head()


,index,Dispatching_base_num,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID,Pickup_DateTime2
0,3271,B02510,2017-06-01 00:00:53,2017-06-01 00:13:22,68.0,107.0,2017-06-01
1,6836,B02835,2017-06-01 00:03:09,2017-06-01 00:16:13,37.0,226.0,2017-06-01
2,10082,B02877,2017-06-01 00:05:14,2017-06-01 00:12:48,225.0,61.0,2017-06-01
3,5816,B02764,2017-06-01 00:05:55,2017-06-01 00:17:49,90.0,48.0,2017-06-01
4,6562,B02765,2017-06-01 00:07:43,2017-06-01 00:30:50,249.0,74.0,2017-06-01


In [4]:
#Cleaning the weather data
Weather_Data['dt_iso'] = pd.to_datetime(Weather_Data['dt_iso'])

Weather_Data = Weather_Data.fillna(0)

Weather_Data.head()


,dt,cc,dt_iso,city_id,city_name,lat,lon,temp,temp_min,temp_max,...,rain_today,snow_1h,snow_3h,snow_24h,snow_today,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1356998400,1,2013-01-01 00:00:00,5128581,"New York City, New York",40.7306,-73.9867,272.03,270.37,273.15,...,0.0,0.0,0.0,0.0,0.0,20,801,Clouds,few clouds,02n
1,1357002000,1,2013-01-01 01:00:00,5128581,"New York City, New York",40.7306,-73.9867,271.46,270.37,272.15,...,0.0,0.0,0.0,0.0,0.0,20,801,Clouds,few clouds,02n
2,1357005600,1,2013-01-01 02:00:00,5128581,"New York City, New York",40.7306,-73.9867,271.19,269.82,272.15,...,0.0,0.0,0.0,0.0,0.0,20,801,Clouds,few clouds,02n
3,1357009200,1,2013-01-01 03:00:00,5128581,"New York City, New York",40.7306,-73.9867,271.07,269.82,272.15,...,0.0,0.0,0.0,0.0,0.0,20,801,Clouds,few clouds,02n
4,1357012800,1,2013-01-01 04:00:00,5128581,"New York City, New York",40.7306,-73.9867,270.83,269.82,272.15,...,0.0,0.0,0.0,0.0,0.0,1,800,Clear,sky is clear,01n


In [5]:
#Cleaning the Zip Data
Zip_Data['LocationID'] = pd.to_numeric(Zip_Data['LocationID'])

Zip_Data = Zip_Data.fillna(0)

Zip_Data.head()


,LocationID,Borough,Zone,service_zone,zipcode
0,1,EWR,Newark Airport,EWR,7114
1,2,Queens,Jamaica Bay,Boro Zone,11366
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone,10462
3,4,Manhattan,Alphabet City,Yellow Zone,10009
4,5,Staten Island,Arden Heights,Boro Zone,10306


In [6]:
#Combining the datasets
Combined_Data = pd.merge(Trip_Data_Sample, Weather_Data, how='left', left_on=['Pickup_DateTime2'], right_on=['dt_iso'])

Combined_Data = pd.merge(Combined_Data, Zip_Data, how='left', left_on=['PUlocationID'], right_on=['LocationID'])

Combined_Data = pd.merge(Combined_Data, Zip_Data, how='left', left_on=['DOlocationID'], right_on=['LocationID'])

Combined_Data = Combined_Data[['Pickup_DateTime', 'DropOff_datetime', 'PUlocationID', 'DOlocationID', 'city_name', 'temp', 'humidity', 'wind_speed', 'weather_id', 'weather_main', 'weather_description', 'Borough_x', 'Zone_x', 'Borough_y', 'Zone_y', 'zipcode_x' , 'zipcode_y']]

Combined_Data.head()


,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID,city_name,temp,humidity,wind_speed,weather_id,weather_main,weather_description,Borough_x,Zone_x,Borough_y,Zone_y,zipcode_x,zipcode_y
0,2017-06-01 00:00:53,2017-06-01 00:13:22,68.0,107.0,"New York City, New York",296.38,68,1,800,Clear,sky is clear,Manhattan,East Chelsea,Manhattan,Gramercy,10024,10024
1,2017-06-01 00:03:09,2017-06-01 00:16:13,37.0,226.0,"New York City, New York",296.38,68,1,800,Clear,sky is clear,Brooklyn,Bushwick South,Queens,Sunnyside,11216,11366
2,2017-06-01 00:05:14,2017-06-01 00:12:48,225.0,61.0,"New York City, New York",296.38,68,1,800,Clear,sky is clear,Brooklyn,Stuyvesant Heights,Brooklyn,Crown Heights North,11216,11216
3,2017-06-01 00:05:55,2017-06-01 00:17:49,90.0,48.0,"New York City, New York",296.38,68,1,800,Clear,sky is clear,Manhattan,Flatiron,Manhattan,Clinton East,10024,10024
4,2017-06-01 00:07:43,2017-06-01 00:30:50,249.0,74.0,"New York City, New York",296.38,68,1,800,Clear,sky is clear,Manhattan,West Village,Manhattan,East Harlem North,10024,10024


In [7]:
#Cleaning the combined data
Combined_Data_Fixed = Combined_Data.rename(columns={'Borough_x': 'Pickup_Location', 'Zone_x': 'Pickup_Neighborhood', 'Borough_y': 'Dropoff_Location', 'Zone_y': 'Dropoff_Neighborhood'})

Combined_Data_Fixed = Combined_Data_Fixed.reset_index()

Combined_Data_Fixed.drop_duplicates(subset=None, keep='first', inplace=False)

Combined_Data_Fixed['Trip_Duration'] = (( pd.to_timedelta(Combined_Data_Fixed.DropOff_datetime).dt.total_seconds() - pd.to_timedelta(Combined_Data_Fixed.Pickup_DateTime).dt.total_seconds() ) / 60).round()

Combined_Data_Fixed = Combined_Data_Fixed.rename(columns={'zipcode_x': 'Pickup_ZipCode', 'zipcode_y': 'Dropoff_ZipCode'})

Combined_Data_Fixed = Combined_Data_Fixed.dropna(subset= ['temp'])

Combined_Data_Fixed['temp'] = pd.to_numeric(Combined_Data_Fixed['temp'])

Combined_Data_Fixed['Temp'] = kelvin_to_fahrenheit(Combined_Data_Fixed['temp']).astype(int)

Combined_Data_Fixed['Pickup_City'] = Combined_Data_Fixed["Pickup_Neighborhood"]+', '+Combined_Data_Fixed["Pickup_Location"]

Combined_Data_Fixed['Dropoff_City'] = Combined_Data_Fixed["Dropoff_Neighborhood"]+', '+Combined_Data_Fixed["Dropoff_Location"]

Combined_Data_Fixed = Combined_Data_Fixed[['Pickup_DateTime', 'DropOff_datetime', 'PUlocationID', 'DOlocationID', 'Pickup_City', 'Dropoff_City', 'Pickup_ZipCode', 'Dropoff_ZipCode', 'Trip_Duration', 'Temp', 'humidity', 'wind_speed', 'weather_id', 'weather_main', 'weather_description']]

Combined_Data_Fixed.index.name = '#'

Combined_Data_Fixed.head()


/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  


,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID,Pickup_City,Dropoff_City,Pickup_ZipCode,Dropoff_ZipCode,Trip_Duration,Temp,humidity,wind_speed,weather_id,weather_main,weather_description
#,,,,,,,,,,,,,,,
0,2017-06-01 00:00:53,2017-06-01 00:13:22,68.0,107.0,"East Chelsea, Manhattan","Gramercy, Manhattan",10024,10024,12.0,74,68,1,800,Clear,sky is clear
1,2017-06-01 00:03:09,2017-06-01 00:16:13,37.0,226.0,"Bushwick South, Brooklyn","Sunnyside, Queens",11216,11366,13.0,74,68,1,800,Clear,sky is clear
2,2017-06-01 00:05:14,2017-06-01 00:12:48,225.0,61.0,"Stuyvesant Heights, Brooklyn","Crown Heights North, Brooklyn",11216,11216,8.0,74,68,1,800,Clear,sky is clear
3,2017-06-01 00:05:55,2017-06-01 00:17:49,90.0,48.0,"Flatiron, Manhattan","Clinton East, Manhattan",10024,10024,12.0,74,68,1,800,Clear,sky is clear
4,2017-06-01 00:07:43,2017-06-01 00:30:50,249.0,74.0,"West Village, Manhattan","East Harlem North, Manhattan",10024,10024,23.0,74,68,1,800,Clear,sky is clear


In [8]:
#Save the Data
os.chdir('/Users/macbook/Desktop/Data-Program-Files/Group-Projects/Team-Folder/Cleaned_RandomSamples/')

Combined_Data_Fixed.to_csv("RideShare_Data_201707.csv", index=True)

print('done')



done
